<a href="https://colab.research.google.com/github/baizhankyzy/female-directors/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stages of EDA

In [4]:
!pip install rdflib
import requests
import json
import rdflib
from rdflib import Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS

# URL of the raw JSON file on GitHub
url = 'https://raw.githubusercontent.com/baizhankyzy/female-directors/refs/heads/main/data1.json'  # Replace with the actual URL

# Fetch the JSON file
response = requests.get(url)
if response.status_code == 200:
    data = response.json()  # Parse JSON data
else:
    raise Exception(f"Failed to fetch JSON file. HTTP Status Code: {response.status_code}")

# Bind namespaces
wd = Namespace("http://www.wikidata.org/entity/")
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")

# Create an empty graph
g = rdflib.Graph()

# Process the JSON data
for entry in data:
    # Extract variables
    director = URIRef(entry['director'])
    film = URIRef(entry['film'])
    release_date = Literal(entry['releaseDate'])
    director_label = Literal(entry['directorLabel'])
    film_label = Literal(entry['filmLabel'])

    # Add triples to the graph
    g.add((director, RDF.type, wd['Q5']))  # Assuming director is a person (Q5 in Wikidata)
    g.add((director, RDFS.label, director_label))
    g.add((director, wdt['P57'], film))  # P57 = directed film
    g.add((film, RDFS.label, film_label))
    g.add((film, wdt['P577'], release_date))  # P577 = publication date

# Serialize the graph to check the results
print(g.serialize(format='turtle'))

Выходные данные были обрезаны до нескольких последних строк (5000).
    ns1:P577 "2001-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q7203905> rdfs:label "Pleasant Moments" ;
    ns1:P577 "2006-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q7204223> rdfs:label "Please Kill Mr. Know It All" ;
    ns1:P577 "2013-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q7206258> rdfs:label "Pocahontas: The Legend" ;
    ns1:P577 "1995-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q72113357> rdfs:label "The Passport" ;
    ns1:P577 "2018-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q72172> rdfs:label "Addiction – When Nothing Is Enough" ;
    ns1:P577 "2004-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q72236475> rdfs:label "The Third Wife" ;
    ns1:P577 "2018-01-01T00:00:00Z",
        "2021-06-10T00:00:00Z" .

<http://www.wikidata.org/entity/Q72314154> rdfs:label "El sueño de Adán" ;
    ns1:P577 "1994-01-01T00:00:00Z" .

<http://www.wikidata.org/entity/Q7235999

## Filter

In [5]:
# Query the graph
query_films_before_2000 = g.query(
    """
    SELECT ?filmLabel ?releaseDate
    WHERE {
        ?film rdfs:label ?filmLabel ;
              <http://www.wikidata.org/prop/direct/P577> ?releaseDate .
        FILTER (STR(?releaseDate) < "2000-01-01T00:00:00Z")
    }
    ORDER BY ?releaseDate
    LIMIT 10
    """
)

# Print the results
for film_label, release_date in query_films_before_2000:
    print(f"{film_label} - {release_date}")

Renaissance Man - 1994-01-01T00:00:00Z
The Promise - 1994-01-01T00:00:00Z
Black Beauty - 1994-01-01T00:00:00Z
Priest - 1994-01-01T00:00:00Z
The Little Rascals - 1994-01-01T00:00:00Z
Corrina, Corrina - 1994-01-01T00:00:00Z
I Like It Like That - 1994-01-01T00:00:00Z
Embrace of the Vampire - 1994-01-01T00:00:00Z
Kommt Mausi raus?! - 1994-01-01T00:00:00Z
Dreamplay - 1994-01-01T00:00:00Z
